### Manually Download Code

In [1]:
# Required Libraries, do not touch
import sys
import os
from configparser import ConfigParser
from datetime import datetime, timedelta

# Add other libraries below
# import numpy
# import pandas

In [2]:
# Variables [Do not change]
manual = True

# Variables [You can change]
startdate = '2002-06-01' # For end date, get the current date, minus 1 day, in string format YYYY-MM-DD
enddate = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

In [3]:
sys.path.append('./main/')
from bmnp import BMNP_Data, BMNP_Download

data = BMNP_Data(startdate=startdate, enddate=enddate, manually=manual)
download = BMNP_Download(manually=manual)

[13:14:33] Dates Missing from SST Database: ['2024-08-05', '2024-08-06', '2024-08-07', '2024-08-08', '2024-08-09', '2024-08-10', '2024-08-11']


In [4]:
missing_dates = ['2024-08-05']

data.download.downloaded_dates = missing_dates
data.delete_singles = False
data.dates_missing = True
data.download.downloaded_dates

['2024-08-05']

In [6]:
command = data.download.setCommand('2024-08-05')

data.download.downloadData('2024-08-05', command)

[13:16:15] Downloading the Date: 2024-08-05. Please give this a moment...
[13:17:28] Files for the date 2024-08-05 were downloaded. Now fixing the data...
[13:17:44] Date: 2024-08-05 has been downloaded and refined.


In [5]:
data.createDHWs()

[12:31:47] Date Index: 8072
[12:31:47] File [2024-07-07] has been created for single-day DHWs.


In [6]:
# Open first item in the "./data/nc_sst/" folder
first_item = os.listdir('./data/nc_sst/')[0]
first_item

'2021-10-23.nc'

In [7]:
# Load this using netcdf4
import netCDF4 as nc
nc_data = nc.Dataset('./data/nc_sst/' + first_item, 'r')
nc_data

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): lon(27), lat(34), time(1)
    variables(dimensions): float32 lon(lon), float32 lat(lat), float32 time(time), float32 analysed_sst(time, lat, lon)
    groups: 